In [12]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class ColorPickerApp:
    def __init__(self, root):
        """
        Initializes the ColorPickerApp.

        Args:
            root (tk.Tk): The root Tkinter window.
        """
        self.root = root
        self.root.title("Image Color Picker")

        # Variables to store image data
        self.image_path = None
        self.image = None 
        self.tk_image = None 
        self.original_image_width = 0
        self.original_image_height = 0

        # Button to open an image file
        self.open_button = tk.Button(root, text="Open Image", command=self.open_image_dialog)
        self.open_button.pack(pady=10)

        
        # Initial size, will adjust based on image or window
        self.canvas = tk.Canvas(root, bg="lightgray", width=600, height=400, relief="sunken", borderwidth=2)
        self.canvas.pack(expand=True, fill="both") # Allow canvas to expand with window
        self.canvas.bind("<Button-1>", self.on_image_click) # Bind left-click event

        # Label to display color information (RGB and Hex)
        self.color_info_label = tk.Label(root, text="Click on an image to get color info.", font=("Arial", 12, "bold"), fg="blue")
        self.color_info_label.pack(pady=10)

        # Label to display pixel coordinates
        self.pixel_coords_label = tk.Label(root, text="Coordinates: (N/A, N/A)", font=("Arial", 10))
        self.pixel_coords_label.pack()

        # Bind configure event to canvas to handle resizing
        self.canvas.bind("<Configure>", self.on_canvas_resize)




    def open_image_dialog(self):
        
        selected_path = filedialog.askopenfilename(
            title="Select an image file",
            filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.gif;*.bmp"), ("All files", "*.*")]
        )
        if selected_path:
            self.open_image_direct(selected_path) # Use the direct loader

    def open_image_direct(self, path):
        """
        Loads and displays an image directly from a given path, without a file dialog.
        Used for initial load or programmatic loading.
        """
        self.image_path = path
        if self.image_path:
            try:
                # Open the image using PIL and convert to RGB (for consistent pixel data)
                self.image = Image.open(self.image_path).convert('RGB')
                self.original_image_width, self.original_image_height = self.image.size

                self.display_image_on_canvas() # Call helper to display and resize

                self.color_info_label.config(text="Image loaded. Click a pixel.")
                self.pixel_coords_label.config(text="Coordinates: (N/A, N/A)")
            except Exception as e:
                # Handle errors during image opening
                messagebox.showerror("Error", f"Failed to open image: {e}")
                self.image = None
                self.tk_image = None
                self.canvas.delete("all") # Clear canvas
                self.color_info_label.config(text="Click on an image to get color info.")
                self.pixel_coords_label.config(text="Coordinates: (N/A, N/A)")

    def display_image_on_canvas(self):
        """
        Resizes the image to fit the current canvas dimensions and displays it.
        Called on initial load and canvas resize.
        """
        if self.image is None:
            return

        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()

        if canvas_width <= 0 or canvas_height <= 0: # Ensure canvas has valid dimensions
            return

        # Calculate aspect ratio to fit image within canvas without distortion
        original_aspect_ratio = self.original_image_width / self.original_image_height
        canvas_aspect_ratio = canvas_width / canvas_height

        if original_aspect_ratio > canvas_aspect_ratio:
            # Image is wider than canvas, scale by width
            new_width = canvas_width
            new_height = int(new_width / original_aspect_ratio)
        else:
            # Image is taller than canvas, scale by height
            new_height = canvas_height
            new_width = int(new_height * original_aspect_ratio)

        # Resize the image using LANCZOS filter for high quality
        display_image = self.image.resize((new_width, new_height), Image.LANCZOS)
        self.tk_image = ImageTk.PhotoImage(display_image)
        
        self.canvas.delete("all") # Clear any previous drawings on the canvas
        
        # Center the image on the canvas
        self.canvas.create_image(canvas_width / 2, canvas_height / 2, image=self.tk_image, anchor="center")

    def on_canvas_resize(self, event):
        """
        Event handler for when the canvas is resized.
        Redraws the image to fit the new dimensions.
        """
        self.display_image_on_canvas()

    def on_image_click(self, event):
   
        if self.image is None:
            messagebox.showwarning("No Image", "Please open an image first to pick a color.")
            return

        # Get current dimensions of the canvas and the displayed image
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        if not self.tk_image: # Ensure an image is actually displayed
            return

        img_on_canvas_width = self.tk_image.width()
        img_on_canvas_height = self.tk_image.height()

        # Calculate the top-left corner of the displayed image on the canvas
        # This is needed because the image is centered
        img_x_start = (canvas_width - img_on_canvas_width) / 2
        img_y_start = (canvas_height - img_on_canvas_height) / 2

        # Adjust event coordinates relative to the top-left of the displayed image
        relative_x_on_display = event.x - img_x_start
        relative_y_on_display = event.y - img_y_start

        # Scale these relative coordinates back to the original image's dimensions
        # This gives us the precise pixel in the original, full-resolution image
        if img_on_canvas_width > 0 and img_on_canvas_height > 0: # Avoid division by zero
            original_pixel_x = int(relative_x_on_display * (self.original_image_width / img_on_canvas_width))
            original_pixel_y = int(relative_y_on_display * (self.original_image_height / img_on_canvas_height))
        else:
            # Fallback or error if image dimensions on canvas are zero
            self.color_info_label.config(text="Error: Image not scaled correctly.")
            return

        # Check if the calculated original pixel coordinates are within the bounds of the original image
        if 0 <= original_pixel_x < self.original_image_width and \
           0 <= original_pixel_y < self.original_image_height:
            try:
                # Get the RGB pixel value from the original PIL Image object
                pixel_rgb = self.image.getpixel((original_pixel_x, original_pixel_y))
                
                # Convert RGB to hexadecimal format (e.g., #RRGGBB)
                hex_color = f"#{pixel_rgb[0]:02X}{pixel_rgb[1]:02X}{pixel_rgb[2]:02X}"
                
                # Update the UI labels with the color information and coordinates
                self.color_info_label.config(text=f"RGB: {pixel_rgb} | Hex: {hex_color}")
                self.pixel_coords_label.config(text=f"Coordinates: ({original_pixel_x}, {original_pixel_y})")
            except Exception as e:
                # Catch any other unexpected errors during pixel retrieval
                self.color_info_label.config(text=f"Error getting pixel: {e}")
        else:
            # Inform the user if they clicked outside the actual image area (but within canvas)
            self.color_info_label.config(text="Clicked outside image boundaries.")
            self.pixel_coords_label.config(text=f"Coordinates: (Out of image bounds)")


# --- Main execution block ---
# This ensures the app runs only when the script is executed directly
if __name__ == "__main__":
    root = tk.Tk() # Create the main Tkinter window
    app = ColorPickerApp(root) # Instantiate the application
    root.mainloop() # Start the Tkinter event loop (keeps the window open)
